In [ ]:
import CompDoobTransform as cdt
import time
import math
import torch
import numpy as np
import matplotlib.pyplot as plt
from CompDoobTransform.utils import negative_binomial_logpdf
from torch.distributions.gamma import Gamma
plt.style.use('ggplot')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Computing on ' + str(device))

In [ ]:
# tuning parameters
T = torch.tensor(1.0, device = device) # time interval
M = 50 # number of time steps
filename = 'logistic.pt'

In [ ]:
# dict for objects relating to latent state process
state = {}

# dimension of state 
d = 1 
state['dim'] = d

# model parameters 
theta = torch.tensor([2.397, 4.429e-03, 0.840, 17.631], device = device)

# drift of diffusion (after Lamperti transformation)
b_constant = theta[0] / theta[2] 
b_factor = theta[1] / theta[2]
b = lambda x: b_constant - b_factor * torch.exp(theta[2] * x) # drift
state['drift'] = b

# diffusion coefficient of diffusion (after Lamperti transformation)
sigma = torch.tensor(1.0, device = device) # diffusion coefficient
state['sigma'] = sigma

# stationary distribution (before Lamperti transformation)
alpha = 2.0 * (0.5 * theta[2]**2 + theta[0]) / theta[2]**2 - 1.0
beta = 2.0 * theta[1] / theta[2]**2
stationary_distribution = Gamma(alpha, beta)

# simulate initial states (from stationary distribution)
initial = lambda N: torch.log(stationary_distribution.sample((N, 1))) / theta[2]
state['initial'] = initial

# time interval
state['terminal_time'] = T


In [ ]:
# dict for objects relating to observations
obs = {}

# dimension of observation
p = 1
obs['dim'] = p

# log-observation density
obs_log_density = lambda x, y: negative_binomial_logpdf(y, theta[3], torch.exp(theta[2] * x)) 
obs['log_density'] = obs_log_density

# simulate observations
def simulate_obs(x, theta3):
    size = theta3.numpy()
    prob = (theta3 / (theta3 + torch.exp(theta[2] * x))).numpy()
    out = torch.tensor(np.random.negative_binomial(n = size, p = prob))
    return out

observation = lambda N: simulate_obs(initial(N))
obs['observation'] = observation

In [ ]:
# learning standardization
N_large = 100000
X = initial(N_large)
Y = torch.tensor(observation(N_large), dtype = torch.float32)

# means and standard deviations
standardization = {'x_mean': torch.mean(X, 0), 
                   'x_std': torch.std(X, 0), 
                   'y_mean': torch.mean(Y, 0), 
                   'y_std': torch.std(Y, 0)}

standardization

In [ ]:
# V0 and Z neural network configuration
V0_net_config = {'layers': [16], 'standardization': standardization}
Z_net_config = {'layers': [d+16], 'standardization': standardization}
net_config = {'V0': V0_net_config, 'Z': Z_net_config}

# learning type
# learning_type = 'standard'
learning_type = 'iterative'                

# create model instance
model = cdt.core.model(state, obs, M, net_config, device = 'cpu')

In [ ]:
# optimization configuration (standard training)
I = 2000
optim_config = {'minibatch': 100, 
                'num_obs_per_batch': 10, 
                'num_iterations': I,
                'learning_rate' : 0.01, 
                'initial_required' : True}
# training
time_start = time.time() 
if learning_type == 'standard':
    model.train_standard(optim_config)
if learning_type == 'iterative':
    model.train_iterative(optim_config)
time_end = time.time()
time_elapsed = time_end - time_start
print("Training time (secs): " + str(time_elapsed))

In [ ]:
# plot loss over optimization iterations
plt.figure()
plt.plot(torch.arange(I), model.loss.to('cpu'), 'k.')
plt.xlabel('iteration', fontsize = 15)
plt.ylabel('loss', fontsize = 15)
plt.show()

In [ ]:
# repeat particle filters
num_obs = [100, 200, 400, 800, 1600]
len_num_obs = len(num_obs)
num_particles = [2**6, 2**7, 2**8, 2**9, 2**10]
R = 100 # number of repeats
BPF = {'ess' : torch.zeros(len_num_obs, R), 'log_estimate' : torch.zeros(len_num_obs, R)}
APF = {'ess' : torch.zeros(len_num_obs, R), 'log_estimate' : torch.zeros(len_num_obs, R)}

for i in range(len_num_obs):
    # number of observations
    K = num_obs[i]

    # number of particles
    N = num_particles[i]

    # simulate latent process and observations
    X0 = initial(1)
    X = torch.zeros(K+1, d)
    X[0,:] = X0.clone()
    Y = torch.zeros(K, p)
    for k in range(K):
        X[k+1,:] = model.simulate_diffusion(X[k,:])
        Y[k,:] = simulate_obs(X[k+1,:])

    for r in range(R):
        # run particle filters
        BPF_output = model.run_BPF(X0.repeat((N,1)), Y, N)
        APF_output = model.run_APF(X0.repeat((N,1)), Y, N)

        # save average ESS%
        BPF_ESS = torch.mean(BPF_output['ess'] * 100 / N)
        APF_ESS = torch.mean(APF_output['ess'] * 100 / N)
        BPF['ess'][i,r] = BPF_ESS
        APF['ess'][i,r] = APF_ESS

        # save log-likelihood estimates
        BPF_log_estimate = BPF_output['log_norm_const'][-1]
        APF_log_estimate = APF_output['log_norm_const'][-1]
        BPF['log_estimate'][i,r] = BPF_log_estimate
        APF['log_estimate'][i,r] = APF_log_estimate

        # print output
        print('No. of observations: ' + str(K) + ' Repeat: ' + str(r)) 
        print('BPF ESS%: ' + str(BPF_ESS))
        print('APF ESS%: ' + str(APF_ESS)) 
        print('BPF log-estimate: ' + str(BPF_log_estimate))
        print('APF log-estimate: ' + str(APF_log_estimate))

# save results
results = {'BPF' : BPF, 'APF' : APF}
torch.save(results, filename)